# Importing Modules, Getting Web Browser Running, and Importing Ticker Info

To begin, we import the modules that will be used for this project. The modules used are:
- csv: used to read and write csv files
- bs4/BeautifulSoup: used to parse information from html documents
- datetime: used for functions for datetime format objects
- pandas: used for data analysis functions and the DataFrame object
- requests: used to fetch data from web pages
- selenium
  - webdriver: allows for controlling of a web browser by python
  - common.by: used with the webdriver code to tell the code what sort of attribute to look for (such as the CSS selector)
  - support.expected_conditions: used with support.ui to define a condition that should be present on the web page (in this analysis, the condition was the prescene of tables on the page)
  - support.ui: used with support.expected_conditions to tell the webdriver to wait for a specified amount of time while searching for an expected condition
  - edge.service: specifies the specific web browser to be used (Edge) and configures the driver for the browser's settings
- time: used to call the sleep function to tell the code to wait specific amounts of time, allowing web pages to load

In [1]:
# Importing modules
import csv
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import requests
from selenium import webdriver
import selenium.webdriver.common.by as By
import selenium.webdriver.support.expected_conditions as EC
import selenium.webdriver.support.ui as WebDriverWait
import selenium.webdriver.edge.service as Service
import time

Next we configure the webdriver that will be used to go to the site and pull our data.

In [2]:
# Getting the webdriver up and running
# Specifying the path to msedgedriver.exe
driver_path = r"C:\Program Files\msedgedriver.exe"

# Creating a service object
service = Service(executable_path=driver_path)

# Initializing the Edge browser
driver = webdriver.Edge(service=service)

We define the url to go to for logging in to estimize and then tell the driver to go to that url.

In [4]:
# Logging into Estimize using the Edge browser
estimize_url = 'https://www.estimize.com/users/sign_in'

driver.get(estimize_url)

We created a csv file that listed the tickers to be analyzed and defined the ticker urls after reviewing the url pattern for companies on the site.
After creating the csv, we imported it into the ticker_list object.

In [4]:
# Importing the ticker list and URLs to their pages
ticker_list = pd.read_csv('ticker_list.csv')
ticker_list.head()

,ticker,URL
0,AMZN,https://www.estimize.com/AMZN
1,AAPL,https://www.estimize.com/AAPL
2,MSFT,https://www.estimize.com/MSFT
3,GOOG,https://www.estimize.com/GOOG
4,TSLA,https://www.estimize.com/TSLA


# Pulling Basic Ticker Info from the URLs

We need to pull several pieces of information from each company page. To begin with, we define a list in which to store the data. Next, we iterate over each row in our ticker_list dataframe. For each row, we first go to the url listed. Once at the url, we use find_element to search for CSS_SELECTORs (found using the developer tools on company pages) that point to the relevant pieces of data we need. These CSS selectors are consistent across company pages which is why we're able to iterate through each url and point at the same elements. Once we pull the information, we add the data to our ticker_data list as a disctionary. Finally, we convert our ticker_data list to a dataframe once we've iterated over all of the urls on our list.

In [5]:
# Making an empty list in which to store the scraped data
ticker_data = []


# Looping through the ticker URLs and pulling the general ticker info
# Setting the ticker_code local variable as the already known ticker code and setting the ticker_url to iterate through
for index, row in ticker_list.iterrows():
    ticker_code = row['ticker']
    ticker_url = row['URL']

    # Navigating to each ticker URL
    driver.get(ticker_url)

    # Pulling the needed basic ticker information
    ticker_name = driver.find_element(By.CSS_SELECTOR, '#releases_show > div.main-content-wrapper.is-closed > div:nth-child(2) > div > div:nth-child(1) > div > div > div > div.release-header-information-title-description > p > a').text
    sector = driver.find_element(By.CSS_SELECTOR, '#releases_show > div.main-content-wrapper.is-closed > div:nth-child(2) > div > div:nth-child(1) > div > div > div > p > span:nth-child(1) > a > span').text
    industry = driver.find_element(By.CSS_SELECTOR, '#releases_show > div.main-content-wrapper.is-closed > div:nth-child(2) > div > div:nth-child(1) > div > div > div > p > span:nth-child(2) > a > span').text
    followers = driver.find_element(By.CSS_SELECTOR, '#summary-stats > div > div > div:nth-child(1) > div.estimize-row.summary-sub-title').text
    analysts = driver.find_element(By.CSS_SELECTOR, '#summary-stats > div > div > div:nth-child(2) > a').text

    # Placing the pulled data into the ticker_data list as a dictionary
    ticker_data.append({
        'Ticker': ticker_code,
        'Name': ticker_name,
        'Sectors': sector,
        'Industries': industry,
        'Followers': followers,
        'Analysts': analysts
    })

# Making a DataFrame out of ticker_data
ticker_df = pd.DataFrame(ticker_data)

# Printing the new DataFrame
print(ticker_df)

   Ticker                          Name                 Sectors  \
0    AMZN               Amazon.com Inc.  Consumer Discretionary   
1    AAPL                    Apple Inc.  Information Technology   
2    MSFT         Microsoft Corporation  Information Technology   
3    GOOG                 Alphabet Inc.  Information Technology   
4    TSLA                    Tesla Inc.  Consumer Discretionary   
5     JNJ             Johnson & Johnson             Health Care   
6      PG          Procter & Gamble Co.        Consumer Staples   
7    NVDA            NVIDIA Corporation  Information Technology   
8    CSCO           Cisco Systems, Inc.  Information Technology   
9    BABA    Alibaba Group Holding Ltd.  Consumer Discretionary   
10     HD          The Home Depot, Inc.  Consumer Discretionary   
11   BIDU                   Baidu, Inc.  Information Technology   
12    WMT          Wal-Mart Stores Inc.        Consumer Staples   
13    CRM                Salesforce.com  Information Technolog

In [6]:
# Saving ticker_df as a csv file
ticker_df.to_csv('ticker_data.csv', index=False)

# Pulling the EPS Data

Next, we need to pull quarterly summary EPS data for each company. The EPS data is stored in a table on the company page, but the table needs to be centered on a specific quarter (q2 2021) to properly display the relevant quarters. To navigate to this table, the pattern of the url is slightly different, but luckily the urls all still include the ticker name, so we are able to iterate over them without too much extra hassle.
For this data pull, we again create a new empty list then navigate to the url. After there, we tell the code to wait three seconds to allow the table to load. After the three seconds, we tell the driver to go to the table based on its CSS selector and then store the entire table in an object called 'table.' We then use the pandas read_html function to parse the table and convert it to a dataframe. The first row is removed, as it contains data for the logged in user, which is empty. We then reset the row indeces and add a column filled with the ticker name to differentiate the data once it's combined. We add the dataframe to our empty list, and once we've iterated through all urls, we convert our list of dataframes into one combined dataframe.

In [16]:
#Preparing an empty list for EPS data storage
eps_dfs = []

# Pulling the EPS data for each ticker
for index, row in ticker_list.iterrows():
    ticker = row['ticker']
    table_url = f"https://www.estimize.com/{ticker}/fq2-2021?metric_name=eps&chart=table"

    # Navigating to the table URL
    driver.get(table_url)

    time.sleep(3)
    # Selecting the table and storing the data in local variable
    table = driver.find_element(By.CSS_SELECTOR, "#main-content > div:nth-child(2) > div:nth-child(3) > div:nth-child(4) > div > table")
    
    # Putting the table data into a DataFrame
    eps_df = pd.read_html(table.get_attribute('outerHTML'))[0]

    # Removing the first row of the data (user data which is all blank)
    eps_df = eps_df.iloc[1:, :]

    # Reseting the row indeces
    eps_df.reset_index(drop=True, inplace=True)

    # Adding a Ticker columnn to differentiate the tables before combining the dataframe
    eps_df['Ticker'] = ticker
    
    # Putting the table data into the eps_dfs list
    eps_dfs.append(eps_df)

# Concatenating all the DataFrames into a single DataFrame
combined_eps_df = pd.concat(eps_dfs, ignore_index=True)

print(combined_eps_df)

C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\1151411026.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  eps_df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\1151411026.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  eps_df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\1151411026.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  eps_df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\1151411026.py:17: FutureWarning: Passing literal html to 're

            EPS FQ1 '20 FQ2 '20 FQ3 '20 FQ4 '20 FQ1 '21 FQ2 '21 FQ3 '21  \
0      Estimize    0.33    0.11    0.42    0.47    0.58    0.74    0.54   
1          Mean    0.34    0.13    0.39    0.46    0.55    0.70    0.56   
2          High    0.52    0.50    0.62    0.91    0.85    1.25    1.10   
3           Low    0.20    0.02    0.15    0.28    0.28    0.35    0.30   
4      Wall St.    0.31    0.07    0.37    0.36    0.48    0.61    0.44   
..          ...     ...     ...     ...     ...     ...     ...     ...   
297    Wall St.    1.24    1.37    1.12    1.01    1.42    1.51    1.19   
298        High    1.26    1.40    1.22    1.11    1.46    1.58    1.22   
299         Low    1.18    1.32    1.06    0.94    1.29    1.38    1.14   
300     Actuals    1.37    1.42    1.17    1.16    1.63    1.64    1.26   
301  YoY Growth     22%     14%     10%      5%     19%     15%      8%   

    FQ4 '21 FQ1 '22 FQ2 '22 FQ3 '22 FQ4 '22 Ticker  
0      0.22    0.47    0.17    0.26    0.23   

C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\1151411026.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  eps_df = pd.read_html(table.get_attribute('outerHTML'))[0]


In [17]:
# Sending the data to a CSV file
combined_eps_df.to_csv('quarterly_eps_data.csv', index=False)

# Pulling the EPS Estimate Analysis Data

Our next task is to pull individual EPS estimate data from each company for each quarter from 2020 through 2022. To do this, we first need to create a list of urls for all of these quarterly sites. We start by making an empty dataframe with Ticker, Quarter, Year, and URL columns. We then iterate over our ticker list to create our urls. To do this, we iterate through each year and then through each quarter. As we do this, we replace the relevant pieces of the url with the iteration's version of it and then add the revised url to our url dataframe.

In [49]:
# Making a DataFrame for storing the URL data
url_df = pd.DataFrame(columns=['Ticker', 'Quarter', 'Year', 'URL'])

# Preparing the URLs for data pulling
for index, row in ticker_list.iterrows():
    ticker = row['ticker']
    base_url = 'https://www.estimize.com/pg/fq1-2020?metric_name=eps&chart=historical'

    # Iterating over quarters and years
    for year in range(2020, 2023):
        for quarter in ['q1', 'q2', 'q3', 'q4']:
            url = f"{base_url.replace('pg', ticker).replace('q1', quarter).replace('2020', str(year))}"
            new_row = {'Ticker': ticker, 'Quarter': quarter, 'Year': year, 'URL': url}
            url_df = pd.concat([url_df, pd.DataFrame([new_row])], ignore_index=True)

print(url_df)

    Ticker Quarter  Year                                                URL
0     AMZN      q1  2020  https://www.estimize.com/AMZN/fq1-2020?metric_...
1     AMZN      q2  2020  https://www.estimize.com/AMZN/fq2-2020?metric_...
2     AMZN      q3  2020  https://www.estimize.com/AMZN/fq3-2020?metric_...
3     AMZN      q4  2020  https://www.estimize.com/AMZN/fq4-2020?metric_...
4     AMZN      q1  2021  https://www.estimize.com/AMZN/fq1-2021?metric_...
..     ...     ...   ...                                                ...
355     PG      q4  2021  https://www.estimize.com/PG/fq4-2021?metric_na...
356     PG      q1  2022  https://www.estimize.com/PG/fq1-2022?metric_na...
357     PG      q2  2022  https://www.estimize.com/PG/fq2-2022?metric_na...
358     PG      q3  2022  https://www.estimize.com/PG/fq3-2022?metric_na...
359     PG      q4  2022  https://www.estimize.com/PG/fq4-2022?metric_na...

[360 rows x 4 columns]


Next we actually need to go through the quarterly urls and and pull the data from the table. We begin as we have before by making an empty list in which to house the data and iterating through our url dataframe. We encounter a new hurdle to tackle, as if there are more than a small number of estimates, the table only shows the first few and then has a "Show More" button. When clicked, the table loads and displays more estimates plus another "Show More" button. To see all the estimates, you have to keep clicking the "Show More" button until they've all loaded. To navigate this setup, we need to first check for the prescence of the button (remember, if there are only a few estimates, the table loads in its entirety and does not include the "Show More" button). To do this, we use the try, except logic. First, we try to find the button by telling the web driver to search for the button element (denoted by its CSS selector), but importantly, we also use our WebDriverWait function to tell the driver to wait up to ten seconds while searching for the button element before moving to the next step. This wait allows the button to load. If the button is identified, we assign its location to the link_element object and then tell the driver to click it using the click() function. This search for the button and click if found repeats until the button is no longer found because all the data has been loaded. If the button isn't found, we print a message showing that the button wasn't found and then move on to the next step of the iteration.

Now that we are certain our table has fully loaded, we again use a try, except logic to grab the table. It is important we do this, as the table does not necessarily have to exist on the page if there were no estimates made for the company in the specific quarter (in our dataset all the quarters did have estimates). In our try portion, we seek out the table using its CSS selector as we did in the previous scrape, pull its data, put it in a dataframe, and add Ticker, Quarter, and Year columns filled with the iterations' values to ensure we can differentiate the data. The finished dataframe is added to our empty list, and once we've iterated through all urls, we convert our list of dataframes into a single combined dataframe.

In [65]:
# Making an empty list in which to store the newly created dataframes
eps_analyst_dfs_list = []

# Iterating over the URL list and navigating to each URL in the webdriver
for _, row in url_df.iterrows():
    test_url = row['URL']
    driver.get(test_url)

    # Checking to see if the button at the bottom of the desired table that expands the table to all values exists (some tables only have a few entries and do not have this button)
    try:
        # Looking for the button for up to 10 seconds and assigning its link to the link_element variable if it is found
        link_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#estimates > div > a'))
        )
        # Clicking the link to load the full table
        link_element.click()
    except:
        # Printing that the button link was not found as well as the URL of the page to allow for followup investigation to confirm that no button was present
        # Critically, we do not end the for loop if the button is not found, as the lack of the button could simply mean that the table was already completely present on the page when our driver loaded the page
        print(f"Link not found or unable to click for URL: {test_url}")

    # Selecting the table
    try:
        table = driver.find_element(By.CSS_SELECTOR, '#estimates > table')
        
        # Extracting the table data
        table_html = table.get_attribute('outerHTML')

        # Converting the table data to a pandas DataFrame
        df = pd.read_html(table_html)[0]

        # Adding new columns and filling them with identifying values
        df['Ticker'] = row['Ticker']
        df['Quarter'] = row['Quarter']
        df['Year'] = row['Year']

        # Appending the individual DataFrame to the combined DataFrame list created earlier
        eps_analyst_dfs_list.append(df)
        
        print(f"Table found and converted to DataFrame for URL: {test_url}")
    except:
        print(f"Table NOT found for URL: {test_url}")

# Converting the DataFrame list into a DataFrame
combined_eps_analyst_df = pd.concat(eps_analyst_dfs_list, ignore_index=True)

C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


Table found and converted to DataFrame for URL: https://www.estimize.com/AMZN/fq1-2020?metric_name=eps&chart=historical


C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


Table found and converted to DataFrame for URL: https://www.estimize.com/AMZN/fq2-2020?metric_name=eps&chart=historical


C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


...

C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


Link not found or unable to click for URL: https://www.estimize.com/PG/fq3-2022?metric_name=eps&chart=historical
Table found and converted to DataFrame for URL: https://www.estimize.com/PG/fq3-2022?metric_name=eps&chart=historical


C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


Link not found or unable to click for URL: https://www.estimize.com/PG/fq4-2022?metric_name=eps&chart=historical
Table found and converted to DataFrame for URL: https://www.estimize.com/PG/fq4-2022?metric_name=eps&chart=historical


C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


In [66]:
combined_eps_analyst_df

,Chart,Unnamed: 1,Analyst,Rank,Points,Value,Confidence,Last Revised,Ticker,Quarter,Year
0,NaN,NaN,AMZN Reported EarningsAmazon.com Inc.,NaN,NaN,0.25,0.25,04/30/20,AMZN,q1,2020
1,NaN,NaN,Estimize Consensus385 estimates weighted,NaN,NaN,0.33,0.33,04/30/20,AMZN,q1,2020
2,NaN,NaN,Estimize Mean386 estimates averaged,NaN,NaN,0.34,0.34,04/30/20,AMZN,q1,2020
3,NaN,NaN,Wall Street Consensus,NaN,NaN,0.31,0.31,04/30/20,AMZN,q1,2020
4,Show EstimateHighlight analyst in all charts,+ Follow this analyst to receive alerts about ...,FlannerFlanner,1.0,25.0,0.25,5.0,04/20/20,AMZN,q1,2020
...,...,...,...,...,...,...,...,...,...,...,...
39219,Show EstimateHighlight analyst in all charts,+ Follow this analyst to receive alerts about ...,RWDKPDRWDKPD,20.0,-6.0,1.30,5.2,06/22/22,PG,q4,2022
39220,Show EstimateHighlight analyst in all charts,+ Follow this analyst to receive alerts about ...,ForniForni,21.0,-6.0,1.30,5.4,07/23/22,PG,q4,2022
39221,Show EstimateHighlight analyst in all charts,+ Follow this analyst to receive alerts about ...,DrognessDrogness,22.0,-10.0,1.34,2.3,01/21/22,PG,q4,2022
39222,Show EstimateHighlight analyst in all charts,+ Follow this analyst to receive alerts about ...,Analyst_9074721Analyst_9074721,23.0,-14.0,1.37,1.6,12/12/21,PG,q4,2022


In [67]:
combined_eps_analyst_df.to_csv('combined_eps_analysts.csv')

# Pulling Individual User Information

Next, we need to pull information from user profiles. We can't use the usernames that we've already collected, as the site allows users to create usernames that may or may not differ from the username that is used for their profile url. To start, we have to create a list of all users who made estimates in our previous individual estimate pull. We don't want to have duplicate users, so instead of a list, we make an empty set here. The first portion of our iteration is the same as above where we iterate over the quareterly data urls and search for and click the "Show More" button. This whole username set process could have been done with our first iteration, but we split it to a separate step to avoid making our code too complex. 

Once the driver is at the url and has fully loaded the table, we again point to the full table. This time though, we identify the table as an html table and then use BeautifulSoup to parse it. After parsing it, we use BeautifulSoup to find all elements in the table that have the object attribute 'data-user' (the url usernames we're looking for) and put them in a list called user_elements. From that list, we pull just the username portion of the elements and put them in another list called user_values. Finally, these usernames are added to original empty set. By being added to a set, which does not house duplicate values, we effectively get a list of all unique usernames. As shown below, we also pring the updated count of usernames from our set so that we can monitor if our code is running properly as it processes.

In [94]:
# Getting all usernames from desired ticker quarters

# Creating a set in which to house the unique usernames
unique_user_values = set()

# Iterating across the URL list as before
for _, row in url_df.iterrows():
    test_url = row['URL']
    driver.get(test_url)

    # Checking to see if the button at the bottom of the desired table that expands the table to all values exists (some tables only have a few entries and do not have this button)
    try:
        # Looking for the button for up to 10 seconds and assigning its link to the link_element variable if it is found
        link_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#estimates > div > a'))
        )
        # Clicking the link to load the full table
        link_element.click()
    except:
        # Printing that the button link was not found as well as the URL of the page to allow for followup investigation to confirm that no button was present
        # Critically, we do not end the for loop if the button is not found, as the lack of the button could simply mean that the table was already completely present on the page when our driver loaded the page
        print(f"Link not found or unable to click for URL: {test_url}")

    # Selecting the table
    try:
        table = driver.find_element(By.CSS_SELECTOR, '#estimates > table')
        
        # Extracting the table data
        table_html = table.get_attribute('outerHTML')

        # Parsing the data using BeautifulSoup
        soup = BeautifulSoup(table_html, 'html.parser')

        # Finding all elements in the soup object that are data-user elements (URL string to their Estimize profile page)
        user_elements = soup.find_all(attrs={'data-user': True})

        # Adding the elements to a list
        user_values = [element['data-user'] for element in user_elements]

        # Adding the list values to the previously created set, effectively eliminating duplicates
        unique_user_values.update(user_values)

        print(f"{len(unique_user_values)} Total Users found after URL: {test_url}")
    except:
        print(f"Table NOT found for URL: {test_url}")

# Converting the set to a list
final_unique_users = list(unique_user_values)
print(f"{len(final_unique_users)} Total Users found across all URLs")

401 Total Users found after URL: https://www.estimize.com/AMZN/fq1-2020?metric_name=eps&chart=historical
629 Total Users found after URL: https://www.estimize.com/AMZN/fq2-2020?metric_name=eps&chart=historical
746 Total Users found after URL: https://www.estimize.com/AMZN/fq3-2020?metric_name=eps&chart=historical
889 Total Users found after URL: https://www.estimize.com/AMZN/fq4-2020?metric_name=eps&chart=historical
971 Total Users found after URL: https://www.estimize.com/AMZN/fq1-2021?metric_name=eps&chart=historical
1042 Total Users found after URL: https://www.estimize.com/AMZN/fq2-2021?metric_name=eps&chart=historical
1106 Total Users found after URL: https://www.estimize.com/AMZN/fq3-2021?metric_name=eps&chart=historical
1225 Total Users found after URL: https://www.estimize.com/AMZN/fq4-2021?metric_name=eps&chart=historical
1294 Total Users found after URL: https://www.estimize.com/AMZN/fq1-2022?metric_name=eps&chart=historical
1344 Total Users found after URL: https://www.estim

We then use csv to write our list of usernames into a csv file.

In [99]:
# Converting the list of usernames to a CSV
output_csv_file = 'analyst_list.csv'

with open(output_csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['User'])
    writer.writerows([[user] for user in final_unique_users])

We now have to pull data from the user profiles. In this version of the notebook, we pulled 1000 users, as it took ~two hours to process. While possible to pull all 9,000+ users found, we deemed the ~20 hours of computing needed to pull this data to be unnecessary for completion of the project (confirmed to be acceptable by the professor).

Because we have so many users to iterate through, we need to pull all of our user data in one large iteration chunk. This setup isn't ideal, as it leads to a very complicated iteration setup, but we felt it was necessary to set the code up in this way for maximum computing efficiency.

First, we need to pull general user data. We follow a familiar pattern of making an empty list and then iterating through user ursl. At the url, we tell the driver to select several different elements using the CSS selector and then we convert the values found at the locations to text and assign them to objects. These objects are added to a dictionary that contains all the general user data and then the data is added to our empty list. The one complicated value for the general data is Roles. Users can have one, two, or three roles which complicates our driver element selector process. If we just created a code to look at the selectors for the three roles, users that have fewer than three or two roles would raise an exception at the extra element selectors. To get around this issue, we make a list of the role selectors and an empty list for roles. For each user, we iterate over the three selectors in order, try to find them using the driver, and assign the found value to our empty role list if one is found. If the selector is not found (for instance if a user only has one role, and we are now checking the second selector in the iteration) the code raises an exception and responds to the exception by passing. This setup returns a list of roles for each person that is assigned along with the other general user data in the dictionary.

Next, we have to pull data from each of three data tables at the bottom of the page. The issue with these tables is that they do not load until they are present on screen. Additionally, they have hidden rows and a "Show More" button that don't appear until the bottom row of the currently loaded data set is visible on screen. This behavior continues until all of the hidden rows have been loaded.  To get around this issue, we first have to apply a bit of logic. We want the driver to scroll down to where the first table (Stocks Covered) is when it first loops through the code. Once it's loaded and visible, we want it to look at the html code of the table in question for if there are hidden rows still present in the table. If there are hidden rows, we want the driver to scroll down again so that more data will load and the "Show More" button will become available. We want to repeat this process until all data is loaded. To apply the first part of the scroll logic, we created an object called first_attempt and set its value equal to True before beginning the iterations. Next, in the iterator after pulling all of the general user data, we tell our driver that if first_attempt is True (which it always is at the beginning of the iteration due to its value being set outside of the iteration), the driver should scroll down a set distance (found by trial and error) that will show the table, allowing it to load. We then set first_attempt to false and restart our if loop. Now that first_attempt is not True, we continue to the next part of our code. In this set of code, we first scroll down again to load more data that may be hidden and reveal a "Show More" button if there is more data to be loaded. If there is more data to be loaded, the table will have rows labeled as 'linked-row-hidden' it its html format. We tell our code to select the full table in its current state, parse it using BeautifulSoup, search for the presence of the hidden rows. If hidden rows are present, we tell the code to click the "Show More" button that should be present and visible on our screen due to the scrolling done at the beginning of the While loop and then restart the While loop to continue the scrolling and searching for hidden rows. Once there are no more hidden rows, the While loop breaks and we continue on to the next part of the code. Also of note, the users do not necessarily have to have this table on their profile, so if the selector returns an exception, we print that there was an issue pulling the Stocks Covered data for the specific username and then allow the code to continue on. 

At this point in our code, we have the Stocks Covered table selected and parsed using BeautifulSoup with the table assigned to an object called 'soup.' Unfortunately, we ran into issues with having our html parsing modules properly understand this object, so we created a workaround. This workaround changes the soup object to string type and splits it line-by-line. We then reviewed the output of this string and found a specific snippet of string ('profile-tbl-ticker') that identified the row where the ticker value is housed. From there, we figured out that the other values we wanted to pull were a consistent number of lines after the ticker line. To pull the data we want, we iterate over the soup string object searching for our ticker line identifier and pull values found in specific line counts after each line with that ticker line identifier string present. We then place the pulled values into a dictionary, convert the dictionary to a datframe, and then add the dataframe to our list of dataframes. We repeat this strategy to pull the data from the other two tables present on the page, but we remove the first_attempt logic. Launching straight into our While logic allows the driver to scroll down far enough to iniate the table's loading process.

In [255]:
test_users = final_unique_users[100:1100]

# Creating list for the user tables
general_user_data_dicts = []
stocks_covered_dfs = []
pending_estimates_dfs = []
scored_estimates_dfs = []

first_attempt = True

# Iterating over URLs
for test_user in test_users:
    url = f"https://www.estimize.com/users/{test_user}"
    driver.get(url)

    name = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > h1 > a")
    analyst_username = name.text.strip()

    join = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > div.profile-bio-footer > div.profile-activity-stats")
    join_date = join.text.strip()

    confidence = driver.find_element(By.CSS_SELECTOR, "#confidence-wrap > div > div.value")
    analyst_confidence = confidence.text.strip()

    error = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.profile-tab-accuracy.is-selected > div:nth-child(1) > div.profile-stat-secondary")
    error_rate = error.text.strip()

    accuracy = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.profile-tab-accuracy.is-selected > div:nth-child(2) > div.profile-stat-secondary")
    accuracy_percentile = accuracy.text.strip()

    points = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.not-selected.profile-tab-points > div:nth-child(1) > div.profile-stat-secondary")
    points_total = points.text.strip()

    stocks = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-unselectable.profile-tab-last.not-selected.profile-tab-stocks > div:nth-child(1) > div.profile-stat-secondary")
    stocks_total = stocks.text.strip()

    pending = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-unselectable.profile-tab-last.not-selected.profile-tab-stocks > div:nth-child(2) > div.profile-stat-secondary")
    pending_estimates = pending.text.strip()

    selectors = [
    "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(1)",
    "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(2)",
    "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(3)"
    ]
    
    # Initializing an empty list to store the role values
    roles = []
    
    # Iterating through each selector and extracting the general user data
    for selector in selectors:
        try:
            role_element = driver.find_element(By.CSS_SELECTOR, selector)
            role_text = role_element.text
            roles.append(role_text)
        except Exception as e:
            pass
    # Creating a DataFrame with the extracted values
    gud_dict = {
                'Analyst Username': analyst_username, 
                'URL Username': test_user,
                'Join Date': join_date,
                'Roles': roles,
                'Analyst Confidence Score': analyst_confidence,
                'Error Rate': error_rate,
                'Accuracy Percentile': accuracy_percentile,
                'Points': points_total,
                'Stocks': stocks_total,
                'Pending': pending_estimates
                }

    general_user_data_dicts.append(gud_dict)
    
    # Checking for hidden entries in the Stocks Covered table
    if first_attempt:
        driver.execute_script("window.scrollBy(0, 500);")
        first_attempt = False
    else:
        while True:
            try:
                driver.execute_script("window.scrollBy(0, 500);")
                table = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#profile-covered-stocks > div.linked-table'))
                )
                table_html = table.get_attribute('outerHTML')
                soup = BeautifulSoup(table_html, 'html.parser')
                hidden_entries = soup.find_all(class_='linked-row hidden')
    
                if not hidden_entries:
                    break
    
                pagination_link = driver.find_element(By.CSS_SELECTOR, '#profile-covered-stocks > div.pagination-footer')
                pagination_link.click()

            except Exception as e:
                print(f"\033[91mError while pulling Stocks Covered for User: {test_user}\033[0m")
                break
    
    # Converting the html to str and splitting it by lines
    lines = str(soup).splitlines()
    
    # Initializing an empty list to store the data
    sc_data = []
    
    # Extracting data following the pattern of how the values are stored in the table by iterating over lines until it finds the target string to begin the iteration
    for i in range(len(lines)):
        if "profile-tbl-ticker" in lines[i]:
            ticker = lines[i+1].strip()
            reports = lines[i + 6].strip()
            quarters = lines[i + 9].strip()
            points = lines[i + 12].strip()
            pts_per_estimate = lines[i + 15].strip()
            error_rate = lines[i + 18].strip()
            accuracy = lines[i + 21].strip()
    
            sc_data.append({
                'Analyst Username': analyst_username, 
                'URL Username': test_user,
                'Ticker': ticker,
                'Reports': reports,
                'Quarters': quarters,
                'Points': points,
                'Pts/Est': pts_per_estimate,
                'Error Rate': error_rate,
                'Accuracy': accuracy
            })
    
    # Creating a DataFrame from the extracted data
    sc_df = pd.DataFrame(sc_data)

    stocks_covered_dfs.append(sc_df)

    print(f"Stocks Covered Scrape Complete for User: {test_user}")

    # Repeating the whole process for pending estimates
    while True:
        try:
            driver.execute_script("window.scrollBy(0, 500);")
            table = WebDriverWait(driver, 0.5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#profile-pending-estimates > div.linked-table'))
            )
            table_html = table.get_attribute('outerHTML')
            soup = BeautifulSoup(table_html, 'html.parser')
            hidden_entries = soup.find_all(class_='linked-row hidden')

            if not hidden_entries:
                break

            pagination_link = driver.find_element(By.CSS_SELECTOR, '#profile-pending-estimates > div.pagination-footer')
            pagination_link.click()

        except Exception as e:
            print(f"\033[91mError while pulling Pending Estimates for User: {test_user}\033[0m")
            break

    lines = str(soup).splitlines()
    
    pe_data = []
    
    for i in range(len(lines)):
        if "profile-tbl-ticker" in lines[i]:
            ticker = lines[i].strip()
            quarter = f"{lines[i + 3].strip()} {lines[i + 4].strip()}"
            reports = f"{lines[i + 8].strip()} {lines[i + 10].strip()}"
            published = lines[i + 13].strip()
            eps = lines[i + 16].strip()
            revenue = lines[i + 19].strip()
    
            pe_data.append({
                'Analyst Username': analyst_username, 
                'URL Username': test_user,
                'Ticker': ticker,
                'Quarter': quarter,
                'Reports': reports,                
                'Published': published,
                'EPS': eps,
                'Revenue': revenue
            })

    pe_df = pd.DataFrame(pe_data)

    pending_estimates_dfs.append(pe_df)

    print(f"Pending Estimates Scrape Complete for User: {test_user}")

    # Repeating the whole process for scored estimates
    while True:
        try:
            driver.execute_script("window.scrollBy(0, 500);")
            table = WebDriverWait(driver, 0.5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#profile-scored-estimates > div.linked-table'))
            )
            table_html = table.get_attribute('outerHTML')
            soup = BeautifulSoup(table_html, 'html.parser')
            hidden_entries = soup.find_all(class_='linked-row hidden')

            if not hidden_entries:
                break

            pagination_link = driver.find_element(By.CSS_SELECTOR, '#profile-scored-estimates > div.pagination-footer')
            pagination_link.click()

        except Exception as e:
            print(f"\033[91mError while pulling Scored Estimates for User: {test_user}\033[0m")
            break
    

    lines = str(soup).splitlines()
    
    se_data = []
    
    for i in range(len(lines)):
        if "profile-tbl-ticker" in lines[i]:
            ticker = lines[i].strip()
            quarter = f"{lines[i + 3].strip()} {lines[i + 4].strip()}"
            reported = lines[i + 7].strip()
            rank = lines[i + 10].strip()
            eps_points = lines[i + 13].strip()
            revenue_points = lines[i + 16].strip()
            total_points = lines[i + 19].strip()
    
            se_data.append({
                'Analyst Username': analyst_username, 
                'URL Username': test_user,
                'Ticker': ticker,
                'Quarter': quarter,
                'Reported': reported,                
                'Rank': rank,
                'EPS Points': eps_points,
                'Revenue Points': revenue_points,
                'Total Points': total_points
            })

    se_df = pd.DataFrame(se_data)

    scored_estimates_dfs.append(se_df)

    print(f"Scored Estimates Scrape Complete for User: {test_user}")
    print(f"Time: {datetime.now()}")

Stocks Covered Scrape Complete for User: analyst_3842733
Error while pulling Pending Estimates for User: analyst_3842733
Pending Estimates Scrape Complete for User: analyst_3842733
Scored Estimates Scrape Complete for User: analyst_3842733
Time: 2023-11-26 19:51:42.349804
Stocks Covered Scrape Complete for User: analyst_2338096
Error while pulling Pending Estimates for User: analyst_2338096
Pending Estimates Scrape Complete for User: analyst_2338096
Scored Estimates Scrape Complete for User: analyst_2338096
Time: 2023-11-26 19:51:46.337794
Stocks Covered Scrape Complete for User: analyst_3301237
Error while pulling Pending Estimates for User: analyst_3301237
Pending Estimates Scrape Complete for User: analyst_3301237
Scored Estimates Scrape Complete for User: analyst_3301237
Time: 2023-11-26 19:51:50.412801
Stocks Covered Scrape Complete for User: analyst_6873880
Error while pulling Pending Estimates for User: analyst_6873880
Pending Estimates Scrape Complete for User: analyst_6873880


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#confidence-wrap > div > div.value"}
  (Session info: MicrosoftEdge=119.0.2151.72); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7F4A02602+60402]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF7F4987352+253666]
	(No symbol) [0x00007FF7F4759C99]
	(No symbol) [0x00007FF7F47A9FD4]
	(No symbol) [0x00007FF7F47AA10A]
	(No symbol) [0x00007FF7F47E81D7]
	(No symbol) [0x00007FF7F47CBACF]
	(No symbol) [0x00007FF7F479D766]
	(No symbol) [0x00007FF7F47E5111]
	(No symbol) [0x00007FF7F47CB863]
	(No symbol) [0x00007FF7F479C585]
	(No symbol) [0x00007FF7F479B993]
	(No symbol) [0x00007FF7F479CD14]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF7F4BE34A4+1161924]
	(No symbol) [0x00007FF7F481E5F6]
	Microsoft::A

In [256]:
combined_stocks_covered_df = pd.concat(stocks_covered_dfs)
combined_stocks_covered_df.to_csv('combined_stocks_covered_df3.csv', index=False)

combined_pending_estimates_df = pd.concat(pending_estimates_dfs)
combined_pending_estimates_df.to_csv('combined_pending_estimates_df3.csv', index=False)

combined_scored_estimates_df = pd.concat(scored_estimates_dfs)
combined_scored_estimates_df.to_csv('combined_scored_estimates_df3.csv', index=False)

combined_general_user_data_df = pd.DataFrame(general_user_data_dicts)
combined_general_user_data_df.to_csv('combined_general_user_data_df3.csv', index=False)